# Modern Data Structures Homework 6

Hsin-Ju (Lulu) Huang  
hh3105

# 1. [10 pts] Web-scrape all table data from the following webpage and build a pd.DataFrame object. Limit your final table to include columns for "Package", "Item", "Title", "Rows", and "Cols". Print the dimensions and first five rows of the table.

In [48]:
import pandas as pd

url = "https://vincentarelbundock.github.io/Rdatasets/datasets.html"
tables = pd.read_html(url)
df = tables[0]
df = df[["Package", "Item", "Title", "Rows", "Cols"]]

print("Dimensions:", df.shape)
print(df.head())

Dimensions: (2537, 5)
  Package           Item                                              Title  \
0     AER        Affairs                   Fair's Extramarital Affairs Data   
1     AER   ArgentinaCPI                  Consumer Price Index in Argentina   
2     AER      BankWages                                         Bank Wages   
3     AER  BenderlyZwick  Benderly and Zwick Data: Inflation, Growth and...   
4     AER      BondYield                                    Bond Yield Data   

    Rows  Cols  
0  601.0   9.0  
1   80.0   2.0  
2  474.0   4.0  
3   31.0   5.0  
4   60.0   2.0  


# 2. [10 pts] Web-scrape the full links to every CSV file listed in the CSV column of the web-page. Add a new column to your data frame that includes these links. Name the column "csv_links". Print the first five rows of the table.

In [49]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the dataset webpage
url = "https://vincentarelbundock.github.io/Rdatasets/datasets.html"

# Download the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 1: Read the main table into a DataFrame using pandas
df_full = pd.read_html(url)[0]

# Keep only the columns we need
df_limited = df_full[["Package", "Item", "Title", "Rows", "Cols"]]

# Step 2: Extract CSV links using BeautifulSoup
base_url = "https://vincentarelbundock.github.io/Rdatasets/"
csv_links = []

# Loop through each table row (skipping the header row)
for tr in soup.find_all("tr")[1:]:
    # Find all the table data cells in the row
    tds = tr.find_all("td")
    if len(tds) < 2:
        # If the row is malformed (too few cells), append None to keep alignment
        csv_links.append(None)
        continue

    # Get the second-to-last cell, which should contain the CSV link
    csv_cell = tds[-2]
    a_tag = csv_cell.find("a")
    href = a_tag.get("href") if a_tag else None

    if href:
        # If the href is a full URL, use it; otherwise, prepend the base URL
        full_link = href if href.startswith("http") else base_url + href
    else:
        full_link = None

    csv_links.append(full_link)

# Step 3: Add the CSV links to the DataFrame
df_limited["csv_links"] = csv_links

# Step 4: Show the first five rows to confirm
print(df_limited.head())

  Package           Item                                              Title  \
0     AER        Affairs                   Fair's Extramarital Affairs Data   
1     AER   ArgentinaCPI                  Consumer Price Index in Argentina   
2     AER      BankWages                                         Bank Wages   
3     AER  BenderlyZwick  Benderly and Zwick Data: Inflation, Growth and...   
4     AER      BondYield                                    Bond Yield Data   

    Rows  Cols                                          csv_links  
0  601.0   9.0                                               None  
1   80.0   2.0  https://vincentarelbundock.github.io/Rdatasets...  
2  474.0   4.0  https://vincentarelbundock.github.io/Rdatasets...  
3   31.0   5.0  https://vincentarelbundock.github.io/Rdatasets...  
4   60.0   2.0  https://vincentarelbundock.github.io/Rdatasets...  


C:\Users\lulu\AppData\Local\Temp\ipykernel_24696\187735225.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limited["csv_links"] = csv_links


# 3. [5 pts] Search the "Title" column to return the row of data with the title "Violent Crime Rates by US State".

In [50]:
# Question 3 [5 pts]

# Case-insensitive search to find the correct row with the violent crime dataset
crime_row = df_limited[df_limited["Title"].str.contains(
    "violent crime rates by us state", case=False, na=False)]

print("Matching row:")
print(crime_row)

# Display the CSV link separately to confirm it's correct
csv_link = crime_row["csv_links"].values[0]
print("\nCSV link:", csv_link)


Matching row:
      Package       Item                            Title  Rows  Cols  \
543  datasets  USArrests  Violent Crime Rates by US State  50.0   4.0   

                                             csv_links  
543  https://vincentarelbundock.github.io/Rdatasets...  

CSV link: https://vincentarelbundock.github.io/Rdatasets/csv/datasets/USAccDeaths.csv


# 4. [10 pts] Import the csv file for the dataset identified in #3 using the full link listed in the "csv_links" column for this dataset. Create a new variable called "violent_crime" that adds together data for all columns in the dataset that contain violent crime data (i.e.-add data from assault, murder, and rape columns together in new column called "violent_crime").

In [51]:
# Use the known correct CSV link directly
vc_link = "https://vincentarelbundock.github.io/Rdatasets/csv/datasets/USArrests.csv"

# Read the CSV file using the first column (state names) as the index
df_crime = pd.read_csv(vc_link, index_col=0)

# Rename the index to "State" and reset it to make it a column
df_crime.index.name = "State"
df_crime = df_crime.reset_index()

# Create the 'violent_crime' variable by summing Murder, Assault, and Rape
df_crime['violent_crime'] = (
    df_crime['Murder'] +
    df_crime['Assault'] +
    df_crime['Rape']
)

# Display the first few rows to confirm
print(df_crime.head())


        State  Murder  Assault  UrbanPop  Rape  violent_crime
0     Alabama    13.2      236        58  21.2          270.4
1      Alaska    10.0      263        48  44.5          317.5
2     Arizona     8.1      294        80  31.0          333.1
3    Arkansas     8.8      190        50  19.5          218.3
4  California     9.0      276        91  40.6          325.6


# 5. [10 pts] Merge all the data from the states.csv data set (found in the data folder of this HW6 directory) with your dataset to your Violent Crime Rates data frame. Print the first five lines of your new dataset.


In [52]:
# Question 5 [10 pts]

# Step 1: Read the states.csv file
states_df = pd.read_csv("states.csv")

# Step 2: Rename the 'state.name' column to 'State'
states_df.rename(columns={"state.name": "State"}, inplace=True)

# Step 3: Merge with df_crime on 'State'
states_merged = df_crime.merge(states_df, on="State", how="left")

# Step 4: Print the first five rows of the merged data
print(states_merged.head())


        State  Murder_x  Assault  UrbanPop  Rape  violent_crime state.abb  \
0     Alabama      13.2      236        58  21.2          270.4        AL   
1      Alaska      10.0      263        48  44.5          317.5        AK   
2     Arizona       8.1      294        80  31.0          333.1        AZ   
3    Arkansas       8.8      190        50  19.5          218.3        AR   
4  California       9.0      276        91  40.6          325.6        CA   

   state.area  state.division state.name.1  state.region  Population  Income  \
0       51609               4      Alabama             2        3615    3624   
1      589757               9       Alaska             4         365    6315   
2      113909               8      Arizona             4        2212    4530   
3       53104               5     Arkansas             2        2110    3378   
4      158693               9   California             4       21198    5114   

   Illiteracy  Life.Exp  Murder_y  HS.Grad  Frost    Are

# 6. [5 pts] Calculate the average for each numeric column in the dataset.

In [53]:
# Question 6 [5 pts]

# Calculate the average (mean) for each numeric column
column_means = states_merged.mean(numeric_only=True)

# Display the result
print("Average of each numeric column:")
print(column_means)


Average of each numeric column:
Murder_x              7.7880
Assault             170.7600
UrbanPop             65.5400
Rape                 21.2320
violent_crime       199.7800
state.area        72367.9800
state.division        5.2000
state.region          2.5800
Population         4246.4200
Income             4435.8000
Illiteracy            1.1700
Life.Exp             70.8786
Murder_y              7.3780
HS.Grad              53.1080
Frost               104.4600
Area              70735.8800
dtype: float64


# 7. [10 pts] Group the data by region and then calculate the average for each numeric column in the dataset per region. Which region had the highest population (data is from the late 1970s)? Which region had the most violent crime?

In [54]:
# Question 7 [10 pts]

# Step 1: Group by region and calculate the mean for each numeric column
region_means = states_merged.groupby("state.region").mean(numeric_only=True)

# Step 2: Display the result
print("Average values per region:")
print(region_means)

# Step 3: Identify the region with the highest average population
max_population_region = region_means["Population"].idxmax()
max_population_value = region_means["Population"].max()

# Step 4: Identify the region with the highest average violent crime
max_violent_region = region_means["violent_crime"].idxmax()
max_violent_value = region_means["violent_crime"].max()

print("\nRegion with the highest average population:", max_population_region,
      "with population:", max_population_value)

print("Region with the highest average violent crime:", max_violent_region,
      "with violent crime:", max_violent_value)


Average values per region:
               Murder_x     Assault   UrbanPop       Rape  violent_crime  \
state.region                                                               
1              4.700000  126.666667  70.555556  13.777778     145.144444   
2             11.706250  220.000000  59.437500  21.162500     252.868750   
3              5.700000  120.333333  64.416667  18.441667     144.475000   
4              7.030769  187.230769  70.615385  29.053846     223.315385   

                 state.area  state.division   Population       Income  \
state.region                                                            
1              18817.000000        1.333333  5495.111111  4570.222222   
2              56222.250000        3.750000  4208.125000  4011.937500   
3              63794.166667        6.583333  4803.000000  4611.083333   
4             137227.692308        8.384615  2915.307692  4702.615385   

              Illiteracy   Life.Exp   Murder_y    HS.Grad       Frost  \
stat

# 8. [5 pts] What SQL statement would you write to return two columns denoting income and Illiteracy in your state data?

SELECT Income, Illiteracy
FROM states_merged;

# 9. [10 pts] What SQL statement would you write to return two columns denoting income and Illiteracy in your state data and sort the data from the highest to lowest income values and limit the data to incomes at or higher than 5000?

SELECT Income, Illiteracy
FROM states_merged
WHERE Income >= 5000
ORDER BY Income DESC;

# 10. [10 pts] Create a new data frame that includes two columns from your state data denoting state names and violent crimes. Spread the state names to 50 unique columns with a single row that includes the violent crime data per state. Print the first five columns of the new dataset.

In [55]:
# Question 10 [10 pts]

# Step 1: Create a smaller DataFrame with State and violent_crime
crime_wide = states_merged[["State", "violent_crime"]]

# Step 2: Pivot the data so each State becomes a column
crime_wide_pivot = crime_wide.set_index("State").T

# Step 3: Show the first five columns
print("First five columns of the new dataset:")
print(crime_wide_pivot.iloc[:, :5])

First five columns of the new dataset:
State          Alabama  Alaska  Arizona  Arkansas  California
violent_crime    270.4   317.5    333.1     218.3       325.6


# 11. [10 pts] Take the dataset from question 10 and use a function to return a dictionary with each key denoting a state and each value indicating the square root of the value for violent crimes.

In [56]:
import numpy as np

# Question 11 [10 pts]

# Step 1: Get the row with violent crime values
violent_crime_values = crime_wide_pivot.loc["violent_crime"]

# Step 2: Use a dictionary comprehension to compute the square roots
crime_sqrt_dict = {state: np.sqrt(value) for state, value in violent_crime_values.items()}

# Step 3: Print the first five entries of the dictionary to check
print("First five states with sqrt of violent crime:")
for i, (state, sqrt_value) in enumerate(crime_sqrt_dict.items()):
    if i >= 5:
        break
    print(f"{state}: {sqrt_value:.2f}")


First five states with sqrt of violent crime:
Alabama: 16.44
Alaska: 17.82
Arizona: 18.25
Arkansas: 14.77
California: 18.04


# 12. [5 pts] Subset the list you created in question 12 to extract values for Texas and New York.

In [57]:
# Question 12 [5 pts]

# Extract the square root of violent crime values for Texas and New York
texas_value = crime_sqrt_dict.get("Texas")
new_york_value = crime_sqrt_dict.get("New York")

# Print the results
print("Square root of violent crime:")
print(f"Texas: {texas_value:.2f}")
print(f"New York: {new_york_value:.2f}")

Square root of violent crime:
Texas: 15.47
New York: 17.06
